# **Trusted** : BCB raw data

A camada trusted como a primeira representação estruturada e validada dos dados após a ingestão bruta (raw). Ela transforma arquivos heterogêneos (JSON, CSV, XML, ZIP etc.) em datasets tabulares padronizados, tipados e auditáveis, mantendo fidelidade à fonte original.

***Se o Raw é evidência, a Trusted é evidência organizada.***

Ela ainda não aplica regras de negócio complexas ou integra múltiplas fontes — isso pertence às camadas seguintes (Refined, Feature, etc.). A Trusted apenas garante que os dados:

- possuem tipos corretos
- têm schema estável
- são consistentes e reprocessáveis
- mantêm rastreabilidade completa

## Raw Data: *SGS*

### *1. Tipo e Estrutura*
Os dados extraídos da SGS são até o momento especificamente as séries SELIC (432) e IPCA (433). Por definição esses são dados diários extraídos (para IPCA o BCB aplica um interpolação para retornar o variação percentual do dia) cujo o principal interesse é *data* e *valor*. Afim de preservar uma estrutura auditável também serão gerados campos que indiquem:

- Data de Processamento (injestion)
- Arquivo Raw que gerou aquele dados
- Hash do conteúdo para governança, pois:
    - Auditável permitindo reconstruir histórico, provar integridade e fazer reconciliação
    - Caso reprocesse o mesmo período os dados não são reinseridos (indepotência)
    - Detectar alteração silenciosa (Se houver alteração de dados antigos a coluna vai acusar)
    - Unicidade Real; Pequenas variações de campo e formatação (como tipo) não impactam a base

tornando assim os dados *raw* uma *tabela clean e auditável". Com isso em mente podemos montar o schema desses dados como:

**Raw:**
 data | valor |
|:------------:|:-------:|
| str | str |

**Trusted**

| series_id | ref_date | value | raw_file | raw_hash | record_hash | ijestioningestion_ts_utc |
|:---------:|:--------:|:-----:|:--------:|:--------:|:-----------:|:------------------------:|
| int | date | float | str | str | str | str |

series_id e ref_date fornecem naturalmente uma chave única para a base de dados.

#### *Classe Modelo*

Como estamos falando de um processo fixo de injestão de dados que serão transformados em arquivos *.parquet* é interessenate construir uma classe fixa de formatação de dados não dependendo de pandas. Esse tipo de arquitetura é interessante inclusive pela restrição de flexbilidade do objeto gerando flexibilidade de código, já que seu futuramente pandas não for mais uma opção o domínio SgsPoint continua o mesmo, com controle de tipo e testabilidade.

```python models.py
@dataclass(frozen=True)
class SgsPoint:
    series_id: int
    ref_date: date
    value: Optional[float]  # pode ser None se vier vazio
    raw_file: str
    raw_hash: str
    record_hash: str
    ingestion_ts_utc: str 
```


Apartir da forma que os arquivos JSON estão sendo salvos podemos facilmente extrair de qual série são aqueles dados.

In [26]:
from pathlib import Path
import json

path = Path("C:\\Users\\Dell\\OneDrive\\Documentos\\GitHub\\ML-ETTJ26\\data\\01_raw\\bcb\\sgs\\433_01-01-2000_31-12-2008.json")

stem = path.stem
series_str = stem.split("_", 1)[0]
series_str

'433'


Com uma rápida olhada nos dados podemos perceber que as informações extaráidas da API está em uma lista de discionários onde ambas as chaves e valores são strings.

```JSON
[{'data': '01/01/2000', 'valor': '1.00'}, ...]
```


In [28]:
with path.open("r", encoding="utf-8") as f:
    json_f = json.load(f)
json_f[0]

{'data': '01/01/2000', 'valor': '0.62'}

Antes de passar esses valores para trusted eles precisam então ser normalizados:
- Data deve ser formato date, não string
- Valor deve ser valor decimal com quantidade fixa de casas, não string 

(obs: float pode gerar comportamento indesejado em razão da base binária)

In [29]:
from datetime import datetime

s = json_f[0]["data"]
datetime.strptime(s, "%d/%m/%Y").date()

datetime.date(2000, 1, 1)

In [30]:
from decimal import Decimal, ROUND_HALF_UP

s = json_f[0].get("valor")
s = str(s).strip()
s = Decimal(s)
vq = s.quantize(Decimal("0.0000000001"), rounding=ROUND_HALF_UP)
print(vq, s)

0.6200000000 0.62


Tranquilamente conseguimos Gerar agora os valores hash com a segurança de comportamento maior

In [31]:
import hashlib
series_id = int(series_str)
ref_date = datetime.strptime(json_f[0]["data"], "%d/%m/%Y").date()
value_dec = vq

payload = f"{series_id}|{ref_date.isoformat()}|{value_dec}"

record_hash = hashlib.sha256(payload.encode("utf-8")).hexdigest()
record_hash

'072692bcfd6ededdac1377bf9cab985900fca426ef5d8543c6d9a7cdc779fb24'

In [ ]:
h = hashlib.sha256()
with path.open("rb") as f:
    
    for chunk in iter(lambda: f.read(1024 * 1024), b""):
        h.update(chunk)
h.hexdigest()

'2064cccd061cb5ed9f8747e42c3cbbea8637b9542f78a6146547c9c8674f67ef'

In [33]:
from datetime import timezone

ingestion_ts_utc = datetime.now(timezone.utc).replace(microsecond=0).isoformat()
ingestion_ts_utc

'2026-02-16T21:29:27+00:00'

In [35]:
import pandas as pd

sgs_t = pd.read_parquet(r"C:\Users\Dell\OneDrive\Documentos\GitHub\ML-ETTJ26\data\02_trusted\bcb\sgs_daily\sgs_daily.parquet")

In [38]:
sgs_t['series_id'].unique()

array([432, 433])